In [42]:
# download the data
import pandas as pd

df = pd.read_csv('comp1_data/train.csv')
print(df.shape[0])
print(df.shape[1])

print(df.head(5))

27643
3
   Id  Popularity                                       Page content
0   0          -1  <html><head><div class="article-info"> <span c...
1   1           1  <html><head><div class="article-info"><span cl...
2   2           1  <html><head><div class="article-info"><span cl...
3   3          -1  <html><head><div class="article-info"><span cl...
4   4          -1  <html><head><div class="article-info"><span cl...


In [43]:
df.head()

,Id,Popularity,Page content
0,0,-1,"<html><head><div class=""article-info""> <span c..."
1,1,1,"<html><head><div class=""article-info""><span cl..."
2,2,1,"<html><head><div class=""article-info""><span cl..."
3,3,-1,"<html><head><div class=""article-info""><span cl..."
4,4,-1,"<html><head><div class=""article-info""><span cl..."


In [75]:
print(type(df.loc[1,'Page content']))

<class 'str'>


In [84]:
from bs4 import BeautifulSoup

def find_link(content: str)-> int:
    soup = BeautifulSoup(content, 'html.parser')
    link = soup.find_all('a')
    
    return len(link)

def find_img(content: str)-> int:
    soup = BeautifulSoup(content, 'html.parser')
    img = soup.find_all('img')
    
    return len(img)

def find_video(content: str)-> int:
    soup = BeautifulSoup(content, 'html.parser')
    video = soup.find_all('video')
    
    return len(video)

def find_audio(content: str)-> int:
    soup = BeautifulSoup(content, 'html.parser')
    audio = soup.find_all('audio')
    
    return len(audio)

def find_total(content: str)-> int:
    return find_link(content) + find_img(content) + find_video(content) + find_audio(content)

# for test
def find_script(content: str)-> int:
    soup = BeautifulSoup(content, 'html.parser')
    script = soup.find_all('script')
    #print(script)
    
    return len(script)

In [198]:
## 找domain
soup = BeautifulSoup(df.loc[0,'Page content'], 'html.parser')
link = soup.find_all('a')
# print(link)

domain_list = []
for link in soup.find_all('a'):
    print(link.get('href'))
    tmp = link.get('href').split('/')
    if tmp[0] == 'http:':
        domain_list.append(tmp[2])
        
print(domain_list)
        

/publishers/space-com/
http://www.space.com/20591-nasa-asteroid-capture-mission-feasibility.html
http://www.space.com/20606-nasa-asteroid-capture-mission-images.html
http://www.space.com/21609-nasa-asteroid-capture-mission-congress.html
http://www.space.com/20606-nasa-asteroid-capture-mission-images.html
http://www.space.com/20601-animation-of-proposed-asteroid-retrieval-mission-video.html
http://www.space.com/20006-deep-space-missions-private-companies.html
http://www.space.com/19823-russia-meteor-explosion-complete-coverage.html
http://www.space.com/19646-asteroid-2012-da14-earth-flyby-complete-coverage.html
http://www.dvidshub.net/image/707596/ida-and-dactyl#.UcHDQvk4uSo
http://www.space.com/34406-spacexs-musk-says-sabotage-unlikely-cause-of-sept-1-explosion-but-still-a-worry.html
http://www.space.com/34405-proxima-centauri-starspots-stellar-cycle-habitable-planet-alien-life.html
http://www.space.com/34404-china-launches-shenzhou-11-astronauts-to-space-lab.html
http://www.space.com/

In [218]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def tokenizer(content):
    
    ## 找domain
    soup = BeautifulSoup(content, 'html.parser')
    link = soup.find_all('a')

    domain_list = []
    for link in soup.find_all('a'):
        if link.get('href') is not None:
            tmp = link.get('href').split('/')
            if tmp[0] == 'http:':
                domain_list.append(tmp[2])
        
    # print(domain_list)
    # return domain_list
    list_set = set(domain_list)
    return list_set

# print(tokenizer(df.loc[0,'Page content']))

In [219]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.pipeline import Pipeline

# hashvec = HashingVectorizer(n_features=2**10)

pipe = Pipeline([('vect', HashingVectorizer(n_features=2**10, tokenizer = tokenizer)), 
                 ('clf', LogisticRegression(solver = "liblinear"))])

scores = cross_val_score(estimator=pipe, X=df['Page content'], y=df['Popularity'], cv=10, scoring='accuracy')
print('[hashing]accuracy: %.5f %s' % (scores.mean(), scores))


[hashing]accuracy: 0.51445 [0.52024584 0.51103074 0.51447178 0.51410999 0.51085384 0.51483357
 0.51374819 0.52858177 0.51085384 0.50578871]


In [89]:
# for n  in range(df.shape[0]):
img_list = []
link_list = []
video_list = []
audio_list = []
total_list = []

for n  in range(df.shape[0]):
    img_list.append(find_img(df.loc[n,'Page content']))
    link_list.append(find_link(df.loc[n,'Page content']))
    video_list.append(find_video(df.loc[n,'Page content']))
    audio_list.append(find_audio(df.loc[n,'Page content']))
    total_list.append(find_total(df.loc[n,'Page content']))
        
df['# of img'] = img_list
df['# of link'] = link_list
df['# of video'] = video_list
df['# of audio'] = audio_list
df['# of total'] = total_list

In [90]:
df.head(100)

,Id,Popularity,Page content,# of img,# of link,# of video,# of audio,# of total
0,0,-1,"<html><head><div class=""article-info""> <span c...",1,22,0,0,23
1,1,1,"<html><head><div class=""article-info""><span cl...",2,18,0,0,20
2,2,1,"<html><head><div class=""article-info""><span cl...",2,11,0,0,13
3,3,-1,"<html><head><div class=""article-info""><span cl...",1,13,0,0,14
4,4,-1,"<html><head><div class=""article-info""><span cl...",52,16,0,0,68
5,5,-1,"<html><head><div class=""article-info""> <span c...",1,30,0,0,31
6,6,1,"<html><head><div class=""article-info""><span cl...",1,11,0,0,12
7,7,-1,"<html><head><div class=""article-info""><span cl...",2,13,0,0,15
8,8,1,"<html><head><div class=""article-info""><span cl...",1,14,0,0,15
9,9,-1,"<html><head><div class=""article-info""><span cl...",2,13,0,0,15


In [58]:
print(tag_list)

[[1, 0, 0, 22], [2, 0, 0, 18], [2, 0, 0, 11], [0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 30], [0, 0, 0, 0]]


In [37]:
df['# of img'] = 0
df['# of video'] = 0
df['# of link'] = 0
# del df['A']

In [38]:
df.head()

,Id,Popularity,Page content,# of img,# of video,# of link
0,0,-1,"<html><head><div class=""article-info""> <span c...",0,0,0
1,1,1,"<html><head><div class=""article-info""><span cl...",0,0,0
2,2,1,"<html><head><div class=""article-info""><span cl...",0,0,0
3,3,-1,"<html><head><div class=""article-info""><span cl...",0,0,0
4,4,-1,"<html><head><div class=""article-info""><span cl...",0,0,0


In [193]:
from sklearn.model_selection import train_test_split

# X = df[['# of img', '# of video', '# of total','# of audio','# of link']].values
X = df[['# of img','# of link']].values
#X = df[['# of link']].values
#X = df[['# of img']].values
y = df['Popularity']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [194]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

lr = LogisticRegression(C = 10, random_state = 0, solver = "liblinear")
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

#train = accuracy_score(y_pred, y_test)
#print('accuracy-train = %f' % train)

scores = cross_val_score(estimator=lr, X=X, y=y, cv=10, scoring='roc_auc')
print('cross_val_score: %f (+/- %.3f)' % (scores.mean(), scores.std()))


cross_val_score: 0.511631 (+/- 0.008)


In [151]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, roc_curve, auc
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score

knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train, y_train)
# K-Fold CV
scores = cross_val_score(knn,X_train, y_train, cv=5)
print('[%d-NN]\n accuracy: %.5f %s' % (k, scores.mean(), scores))

[100-NN]
 accuracy: 0.50646 [0.50219582 0.50994575 0.50981912 0.50865857 0.50168002]


In [152]:
from sklearn.svm import SVC
svc = SVC(kernel='rbf', random_state=0, gamma=1.0, C=100.0, probability=True)

scores = cross_val_score(estimator=svc, X=X_train, y=y_train, cv=10, scoring='accuracy')
print('svc: %.3f (+/- %.3f)' % (scores.mean(), scores.std()))

KeyboardInterrupt: 

In [121]:
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, VotingClassifier

clf = VotingClassifier(estimators=[('lr', lr), ('dt', knn), ('knn', svc)], voting='soft', weights=[2,1,0])
scores = cross_val_score(estimator=clf, X=X_train, y=y_train, cv=10, scoring='accuracy')
print('voting: %.3f (+/- %.3f)' % (scores.mean(), scores.std()))

voting: 0.509 (+/- 0.011)


In [132]:
# adaboost
ada = AdaBoostClassifier(base_estimator=lr, n_estimators=500)
ada = ada.fit(X_train, y_train)
y_train_pred = ada.predict(X_train)

ada_train = accuracy_score(y_train, y_train_pred)
print('[AdaBoost] accuracy-train = %f,' % ada_train)

[AdaBoost] accuracy-train = 0.510698,


In [148]:
d = df[['Id','# of link']]
d.columns = ['id','Popularity']
d= d.drop
d.to_csv("temp.csv")

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'